In [7]:
import sys
sys.path.append('..')

import json
import pennylane as qml
import pennylane.numpy as np
from src.tests import calculate_heisenberg_fidelity_vs_noise
from src.plots import plot_fidelities_vs_noise, plot_fidelities_vs_noise_changing_depth

Parameters Definition

In [2]:
def block(weights, wires):
    qml.RY(weights[0], wires=wires[0])
    qml.RY(weights[1], wires=wires[1])
    qml.CNOT(wires=wires)

In [3]:
dev = qml.device("lightning.qubit", wires=4)

@qml.qnode(dev)
def circuit(template_weights):
    qml.MPS(
        wires=range(4),
        n_block_wires=2,
        block=block,
        n_params_block=2,
        template_weights=template_weights,
    )
    return qml.expval(qml.PauliZ(wires=2))

In [4]:
def costfunc(params):
    cost = 0
    for i in range(len(params)):
        if i < len(params) / 2:
            cost += circuit(params)
        else:
            cost -= circuit(params)
    return cost

In [5]:
params = np.random.random(size=[3, 2], requires_grad=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)

for k in range(100):
    if k % 20 == 0:
        print(f"Step {k}, cost: {costfunc(params)}")
    params = optimizer.step(costfunc, params)

TypeError: MPS.__init__() missing 1 required positional argument: 'block'

In [8]:
def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.
    Args:
        h (float): magnetic field strength
    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """


    couplings = [-h]
    ops = [qml.PauliX(3)]

    for i in range(3):
        couplings = [-h] + couplings
        ops = [qml.PauliX(i)] + ops   
    
    for i in range(4):
        couplings = [-1] + couplings
        ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops

    return qml.Hamiltonian(couplings,ops)



dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.
    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value
    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """


    for i in range(4):
        qml.RY(params[i], wires=i)
    
    for i in range(4):
        qml.CNOT([i,(i+1)%4])
    for i in range(4):
        qml.RZ(params[i+4], wires=i)
        
    
    for i in range(4):
        qml.RY(params[i+8], wires=i)
    
    for i in range(4):
        qml.CNOT([i,(i+1)%4])
    for i in range(4):
        qml.RZ(params[i+12], wires=i)

    return qml.expval(H)


def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.
    Args:
        h (float): magnetic field strength
    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """


    step_size = 0.02
    opt = qml.GradientDescentOptimizer(stepsize=0.2)
    #opt = qml.QNGOptimizer(stepsize=step_size, approx="block-diag")
    H =  create_Hamiltonian(h)
    theta = np.array([np.pi/2 for i in range(16)], requires_grad=True)
    # Put your code here #
    # store the values of the cost function
    energy = [model(theta, H)]

    # store the values of the circuit parameter
    params = [theta]

    max_iterations = 500
    conv_tol = 1e-06

    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(model, theta, H=H)
        energy.append(model(theta, H))
        params.append(theta)

        conv = np.abs(energy[-1] - prev_energy)

        # if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

        if conv <= conv_tol:
            break
    return theta


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."


test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")


Running test case 0 with input '1.0'...
Step = 0,  Energy = -1.34013799 Ha
Step = 1,  Energy = -2.34886836 Ha
Step = 2,  Energy = -3.02123336 Ha
Step = 3,  Energy = -3.51740041 Ha
Step = 4,  Energy = -3.89557127 Ha
Step = 5,  Energy = -4.16269662 Ha
Step = 6,  Energy = -4.34011268 Ha
Step = 7,  Energy = -4.45804681 Ha
Step = 8,  Energy = -4.53819625 Ha
Step = 9,  Energy = -4.59391304 Ha
Step = 10,  Energy = -4.63410890 Ha
Step = 11,  Energy = -4.66496690 Ha
Step = 12,  Energy = -4.69064341 Ha
Step = 13,  Energy = -4.71379336 Ha
Step = 14,  Energy = -4.73601793 Ha
Step = 15,  Energy = -4.75820404 Ha
Step = 16,  Energy = -4.78075894 Ha
Step = 17,  Energy = -4.80376736 Ha
Step = 18,  Energy = -4.82710042 Ha
Step = 19,  Energy = -4.85049698 Ha
Step = 20,  Energy = -4.87362887 Ha
Step = 21,  Energy = -4.89615424 Ha
Step = 22,  Energy = -4.91775890 Ha
Step = 23,  Energy = -4.93818439 Ha
Step = 24,  Energy = -4.95724260 Ha
Step = 25,  Energy = -4.97481842 Ha
Step = 26,  Energy = -4.99086363 H